## SQL

#### Libraries

In [1]:
import sqlite3

#### SQL syntax

In [2]:
# retrieving info
'''SELECT column_name_1, column_name_2 FROM table_name'''

# filtering
# =: eq
# <>: not equal
# >: gt
# <: lt
# >=: gteq
# <=: lteq
# BETWEEN ... AND ...: range
'''SELECT column_name_1, column_name_2 FROM table_name WHERE price > 10'''

# NULL is empty
'''SELECT column_name_1, column_name_2 FROM table_name WHERE price = NULL'''

# order by ASC or DESC
'''SELECT column_name_1, column_name_2 FROM table_name ORDER BY price ASC'''

# distinct values (also possible to have distinct 2 or more columns)
'''SELECT DISTINCT column_name_1 FROM table_name'''

# regex filtering
'''SELECT column_name_1 FROM table_name WHERE major IN ("Physics", "Mathematics")'''
'''SELECT column_name_1 FROM table_name WHERE major NOT IN ("Computer Science")'''
# like ('_' as only 1 char or '%' as any number of char)
# major starts with C
'''SELECT column_name_1 FROM table_name WHERE major LIKE "C%"'''
# major has C_mputer structure, and any char can be instead of _ (not only 1)
'''SELECT column_name_1 FROM table_name WHERE major LIKE "C_mputer"''' 

# aggregate functions (SUM, MIN, MAX, AVG, COUNT)
'''SELECT SUM(column_name_1) FROM table_name''' 

# group by some column
'''SELECT SUM(column_name_1) FROM table_name GROUP BY customer_id''' 
# group by year and count how many entries for each year
'''SELECT Founded, COUNT(*) FROM t_big10west GROUP BY Founded'''

# filtering after group by
# WHERE applied for original table; HAVING applied for a new table and creating another table
'''SELECT SUM(column_name_1) FROM table_name GROUP BY customer_id HAVING price > 10''' 

# AS to save a temporary variable 
'''SELECT SUM(column_name_1) AS sum_of_smt FROM table_name GROUP BY customer_id HAVING price > 10'''

# join several tables:
# (INNER) JOIN: output table will have entries which are only in both tables at the same time
# LEFT JOIN: output table will have entries which are in left tables
# RIGHT JOIN: output table will have entries which are in right tables
# FULL JOIN: output table will have entries which are in left and right tables
'''SELECT table_1_column_3, table_2_column_7 FROM table_name_1 LEFT JOIN table_name_2 ON table_1_column_4 > 10'''

# creating a table. datatype is optional
'''CREATE TABLE table_name [col1 datatype, col2 datatype]'''
# delete table
'''DROP TABLE table_name'''

# insert new row. [col1, col2] is optional, if giving all values
'''INSERT INTO table_name [col1, col2] VALUES value1, value2'''
# update row. WHERE is mandatory!!! (otherwise all rows will be modified)
'''UPDATE table_name SET col1=value1, col3=value3 WHERE user_i = 10'''
# delete row. WHERE is mandatory!!! (otherwise all rows will be deleted)
'''DELETE FROM table_name WHERE user_i = 10'''

# sqlite_master is in all sqlite databases and contains info about all tables in the db
'''SELECT * FROM sqlite_master'''

'SELECT * FROM sqlite_master'

#### Create SQL table

In [3]:
conn = sqlite3.connect('./help/big10.db')
c = conn.cursor()

# Double quotes are the norm in most of the python programs. 
# Triple quotes are used mostly when you have a big multi-line string 
# copy-pasted from somewhere else and you want to embed them in your program.
c.execute('''CREATE TABLE t_big10west (ID, University, City, State, Founded)''')

universities = [(101, 'University of Illinois', 'Urbana', 'Illinois', 1867),
                (202, 'University of Iowa', 'Iowa City', 'Iowa', 1847),
                (303, 'University of Minnesota', 'Minneapolis', 'Minnesota', 1851),
                (404, 'University of Nebraska', 'Lincoln', 'Nebraska', 1869),
                (505, 'Northwestern University', 'Evanston', 'Illinois', 1851),
                (606, 'Purdue University', 'West Lafayette', 'Indiana', 1869),
                (707, 'University of Wisconsin', 'Madison', 'Wisconsin', 1849)]
c.executemany('INSERT INTO t_big10west VALUES (?,?,?,?,?)', universities)
conn.commit()

# get all rows
for row in c.execute('''SELECT * FROM t_big10west'''):
    print(row)

# print column names
print([desc[0] for desc in c.description])
    
conn.close()

(101, 'University of Illinois', 'Urbana', 'Illinois', 1867)
(202, 'University of Iowa', 'Iowa City', 'Iowa', 1847)
(303, 'University of Minnesota', 'Minneapolis', 'Minnesota', 1851)
(404, 'University of Nebraska', 'Lincoln', 'Nebraska', 1869)
(505, 'Northwestern University', 'Evanston', 'Illinois', 1851)
(606, 'Purdue University', 'West Lafayette', 'Indiana', 1869)
(707, 'University of Wisconsin', 'Madison', 'Wisconsin', 1849)
['ID', 'University', 'City', 'State', 'Founded']


#### Chaning the mistake

In [4]:
conn = sqlite3.connect('./help/big10.db')
c = conn.cursor()

c.execute('UPDATE t_big10west SET Founded=1848 WHERE ID=707')
conn.commit()

conn.close()

#### Get all rows 

In [5]:
conn = sqlite3.connect('./help/big10.db')
c = conn.cursor()

# get all rows
for row in c.execute('SELECT * FROM t_big10west'):
    print(row)
    
conn.close()

(101, 'University of Illinois', 'Urbana', 'Illinois', 1867)
(202, 'University of Iowa', 'Iowa City', 'Iowa', 1847)
(303, 'University of Minnesota', 'Minneapolis', 'Minnesota', 1851)
(404, 'University of Nebraska', 'Lincoln', 'Nebraska', 1869)
(505, 'Northwestern University', 'Evanston', 'Illinois', 1851)
(606, 'Purdue University', 'West Lafayette', 'Indiana', 1869)
(707, 'University of Wisconsin', 'Madison', 'Wisconsin', 1848)


## Chinook database

In [6]:
conn = sqlite3.connect('./help/Chinook_Sqlite.sqlite')
c = conn.cursor()

# count number of tables
n_tables = 0
table_names = []
for table in c.execute('SELECT * FROM sqlite_master'):
    # if it is a table
    if table[0] == 'table':
        n_tables += 1
        table_names.append(table[1])
    
print('Number of tables: ', n_tables)
print('Table names: ', table_names)
    
conn.close()

Number of tables:  11
Table names:  ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


#### Functions with the db

In [7]:
def is_string(value):
    return isinstance(value, str)   

#### Get primary keys of tables that start from a specific character (not case sensitive)

In [8]:
def list_table_ids_by_letter(char):
    '''
        Function takes as a single character "char" and returns a list of primary keys of tables,
        which name starts from character "char" (ignoring cases)
    '''
    
    if not is_string(char):
        raise TypeError('Input should be a type string')
        
    if len(char) != 1:
        raise ValueError('Input should be a single character')
        
    char = char.lower()
    
    # if not alphabetic
    if not char.isalpha():
        raise ValueError('Input should be alphabetic')

    # read the db
    conn = sqlite3.connect('./help/Chinook_Sqlite.sqlite')
    c = conn.cursor()

    table_names = []
    for table in c.execute('SELECT * FROM sqlite_master'):
        # if it is a table
        if table[0] == 'table' and table[1][0].lower() == char:
            # get the location of "constraint" keyword
            constr_loc = table[4].find('CONSTRAINT')
            # add to the list the name of the primary key (which is in a substring before ']' character)
            table_names.append(table[4][constr_loc + 12:].partition(']')[0])

    conn.close()
    
    return table_names
        
list_table_ids_by_letter('a')

['PK_Album', 'PK_Artist']

#### Get primary keys of albums which title starts from a specific character (not case sensitive)

In [9]:
def list_album_ids_by_letter(char):
    '''
        Function takes as a single character "char" and returns a list of primary keys of Album table,
        which album's title starts from character "char" (ignoring cases)
    '''
    
    if not is_string(char):
        raise TypeError('Input should be a type string')
        
    if len(char) != 1:
        raise ValueError('Input should be a single character')
        
    char = char.lower()
    
    # if not alphabetic
    if not char.isalpha():
        raise ValueError('Input should be alphabetic')

    # read the db
    conn = sqlite3.connect('./help/Chinook_Sqlite.sqlite')
    c = conn.cursor()

    album_names = []
    for row in c.execute('SELECT * FROM Album WHERE Title LIKE "' + char + '%" OR Title LIKE "' + char.upper() + '%"'):
        print(row)
        album_names.append(row[0])

    conn.close()
    
    return album_names
        
list_album_ids_by_letter('q')

(86, 'Quanta Gente Veio Ver (Live)', 27)
(87, 'Quanta Gente Veio ver--Bônus De Carnaval', 27)
(262, 'Quiet Songs', 197)


[86, 87, 262]

#### Get primary keys of tracks which album title starts from a specific character (not case sensitive)

In [10]:
def list_song_ids_by_album_letter(char):
    '''
        Function takes as a single character "char" and returns a list of primary keys of Track table,
        whose album names begin with that letter
    '''
    
    if not is_string(char):
        raise TypeError('Input should be a type string')
        
    if len(char) != 1:
        raise ValueError('Input should be a single character')
        
    char = char.lower()
    
    # if not alphabetic
    if not char.isalpha():
        raise ValueError('Input should be alphabetic')

    # read the db
    conn = sqlite3.connect('./help/Chinook_Sqlite.sqlite')
    c = conn.cursor()

    album_names = []
    for row in c.execute('SELECT * FROM Track LEFT JOIN Album ON Track.AlbumId = Album.AlbumId WHERE (Album.Title LIKE "' + char + '%" OR Album.Title LIKE "' + char.upper() + '%")'):
        # print(c.description)
        print(row)
        album_names.append(row[0])

    conn.close()
    
    return album_names
        
list_song_ids_by_album_letter('q')

(1087, 'Introdução (Live)', 86, 1, 7, None, 154096, 5227579, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1088, 'Palco (Live)', 86, 1, 7, None, 238315, 8026622, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1089, 'Is This Love (Live)', 86, 1, 7, None, 295262, 9819759, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1090, 'Stir It Up (Live)', 86, 1, 7, None, 282409, 9594738, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1091, 'Refavela (Live)', 86, 1, 7, None, 236695, 7985305, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1092, 'Vendedor De Caranguejo (Live)', 86, 1, 7, None, 248842, 8358128, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1093, 'Quanta (Live)', 86, 1, 7, None, 357485, 11774865, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1094, 'Estrela (Live)', 86, 1, 7, None, 285309, 9436411, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1095, 'Pela Internet (Live)', 86, 1, 7, None, 263471, 8804401, 0.99, 86, 'Quanta Gente Veio Ver (Live)', 27)
(1096, 'Cérebro Eletrônico (Live)

[1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1104,
 3349,
 3350]

#### Get cost of buying all songs of albums which title starts from a specific character (not case sensitive)

In [11]:
def total_cost_by_album_letter(char):
    '''
        Function takes as a single character "char" and returns a cost of buying all songs from album,
        whose title begins with that letter
    '''
    
    if not is_string(char):
        raise TypeError('Input should be a type string')
        
    if len(char) != 1:
        raise ValueError('Input should be a single character')
        
    char = char.lower()
    
    # if not alphabetic
    if not char.isalpha():
        raise ValueError('Input should be alphabetic')

    # read the db
    conn = sqlite3.connect('./help/Chinook_Sqlite.sqlite')
    c = conn.cursor()

    album_names = []
    for row in c.execute('SELECT SUM(Track.UnitPrice) FROM Track LEFT JOIN Album ON Track.AlbumId = Album.AlbumId WHERE (Album.Title LIKE "' + char + '%" OR Album.Title LIKE "' + char.upper() + '%")'):
        album_names.append(row[0])

    conn.close()
    
    return album_names
        
total_cost_by_album_letter('q')

[19.799999999999997]